# Profiling main skforecast classes and methods

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import series_wide_to_long

from skforecast.exceptions import MissingValuesWarning, MissingExogWarning
from skforecast.exceptions import IgnoredArgumentWarning
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor

%load_ext pyinstrument
%load_ext line_profiler

In [40]:
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import (
    series_wide_dt,
    series_dict_range,
    series_dict_dt,
    exog_wide_range,
    exog_dict_range,
    exog_pred_wide_range,
    exog_pred_long_dt,
    exog_pred_dict_range,
    series_dict_nans_train,
    exog_dict_nans_train,
    exog_dict_nans_test,
    expected_df_to_long_format
)

In [3]:
n = 5000
n_series = 500
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id", sort=False).apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)
display(series)

exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, n_series)]
# exog = [exog_i.copy().sample(frac=0.8).sort_index() for exog_i in exog]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])
display(exog)

# Formato dict
series_dict = {
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

exog_dict = {
    sid: exog.loc[sid]for sid in exog.index.levels[0]
}

value
series_id  datetime         
series_1   2023-01-01     64
           2023-01-02     55
           2023-01-03     59
           2023-01-04      2
           2023-01-05     20
...                      ...
series_499 2036-09-04      2
           2036-09-05     31
           2036-09-06     64
           2036-09-07     62
           2036-09-08     32

[2495000 rows x 1 columns]

exog_1  exog_2  exog_3  exog_4  exog_5  exog_6
series_id  datetime                                                  
series_1   2023-01-01       2       1       5       6       2       5
           2023-01-02       8       1       1       2       4       4
           2023-01-03       1       1       7       4       3       1
           2023-01-04       8       4       4       9       4       7
           2023-01-05       6       8       8       9       3       3
...                       ...     ...     ...     ...     ...     ...
series_499 2036-09-04       9       7       6       8       2       1
           2036-09-05       1       7       6       3       7       1
           2036-09-06       5       9       8       4       9       5
           2036-09-07       4       3       2       9       4       1
           2036-09-08       5       9       1       4       7       6

[2495000 rows x 6 columns]

In [80]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

In [81]:
series_dict = {
    'series_1': pd.Series(np.arange(50), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_2': pd.Series(np.arange(10, 40), index=pd.RangeIndex(start=20, stop=50, step=1)),
    'series_3': pd.Series(np.arange(20, 70), index=pd.RangeIndex(start=0, stop=50, step=1)),
}

In [83]:
forecaster = ForecasterRecursiveMultiSeries(regressor=Ridge(random_state=123), 
                                                          lags=2, transformer_series=None,
                                                          encoding='onehot')
forecaster.fit(series=series_dict)
forecaster.predict(steps=5, levels=None)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:477                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

,level,pred
50,series_1,49.999406
50,series_2,39.999652
50,series_3,69.999396
51,series_1,50.999085
51,series_2,40.999454
51,series_3,70.999070
52,series_1,51.998603
52,series_2,41.999156
52,series_3,71.998580
53,series_1,52.998177


In [84]:
forecaster = ForecasterRecursiveMultiSeries(regressor=Ridge(random_state=123), 
                                                          lags=2, transformer_series=None,
                                                          encoding='onehot')

cv = TimeSeriesFold(
            initial_train_size = 30,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,window_size=forecaster.window_size
         )
cv.split(X=series_dict,as_pandas=True)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:477                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

Information of folds
--------------------
Number of observations used for initial training: 30
Number of observations used for backtesting: 20
    Number of folds: 7
    Number skipped folds: 0 
    Number of steps per fold: 3
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0
    Last fold only includes 2 observations.

Fold: 0
    Training:   0 -- 29  (n=30)
    Validation: 30 -- 32  (n=3)
Fold: 1
    Training:   No training in this fold
    Validation: 33 -- 35  (n=3)
Fold: 2
    Training:   No training in this fold
    Validation: 36 -- 38  (n=3)
Fold: 3
    Training:   No training in this fold
    Validation: 39 -- 41  (n=3)
Fold: 4
    Training:   No training in this fold
    Validation: 42 -- 44  (n=3)
Fold: 5
    Training:   No training in this fold
    Validation: 45 -- 47  (n=3)
Fold: 6
    Training:   No training in this fold
    Validation: 48 -- 49  (n=2)



,fold,train_start,train_end,last_window_start,last_window_end,test_start,test_end,test_start_with_gap,test_end_with_gap,fit_forecaster
0,0,0,30,28,30,30,33,30,33,True
1,1,0,30,31,33,33,36,33,36,False
2,2,0,30,34,36,36,39,36,39,False
3,3,0,30,37,39,39,42,39,42,False
4,4,0,30,40,42,42,45,42,45,False
5,5,0,30,43,45,45,48,45,48,False
6,6,0,30,46,48,48,50,48,50,False


In [94]:
cv = TimeSeriesFold(
            initial_train_size = 30,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,
            window_size=forecaster.window_size, 
            verbose=False
         )
metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster            = forecaster,
                                            series                = series_dict,
                                            cv                    = cv,
                                            levels                = 'l1',
                                            metric                = 'mean_absolute_error',
                                            add_aggregated_metric = False,
                                            exog                  = None,
                                            verbose               = False,
                                            n_jobs=1
                                        )

backtest_predictions.head(3)

ValueError: Levels ['l1'] not found in `series`, available levels are ['series_1', 'series_2', 'series_3']. Review `levels` argument.

In [36]:
type(series_dict['series_1'].index)

pandas.core.indexes.range.RangeIndex

In [37]:
isinstance(series_dict['series_1'].index, type(series_dict['series_1'].index))

True

In [38]:
from skforecast.utils import check_preprocess_series



In [42]:
series_dict = {
    'series_1': pd.Series(np.arange(50), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_2': pd.Series(np.arange(10, 40), index=pd.RangeIndex(start=20, stop=50, step=1)),
    'series_3': pd.Series(np.arange(20, 70), index=pd.RangeIndex(start=0, stop=50, step=1)),
}

In [63]:
from skforecast.preprocessing import series_wide_to_long

series = pd.DataFrame({'1': pd.Series(np.arange(10, dtype=float)), 
                           '2': pd.Series(np.arange(10, dtype=float))})
series.index = pd.date_range(start='2000-01-01', periods=len(series), freq='D')
series = series_wide_to_long(series)

exog = pd.DataFrame({'exog_1': np.arange(100, 110, dtype=float),
                        'exog_2': np.arange(1000, 1010, dtype=float)})
exog.index = pd.date_range(start='2000-01-01', periods=len(exog), freq='D')
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, 3)]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])

In [65]:
forecaster = ForecasterRecursiveMultiSeries(
        Ridge(), lags=5, encoding='ordinal_category', transformer_series=None
    )

results = forecaster._create_train_X_y(
    series=series, exog=exog, store_last_window=['3']
) 

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:477                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── MissingExogWarning ─────────────────────────────────╮
│ {'1', '2'} not present in `exog`. All values of the exogenous variables for these    │
│ series will be NaN.                                                                  │
│                                                                                      │
│ Category : MissingExogWarning                                                        │
│ Location : <string>:51                                                               │
│ Suppress : warnings.simplefilter('ignore', category=MissingExogWarning)              │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────── IgnoredArgumentWarning ───────────────────────────────╮
│ Series {'3'} are not present in `series`. No last window is stored for them.         │
│                                                                                      │
│ Category : IgnoredArgumentWarning                                                    │
│ Location : <string>:165                                                              │
│ Suppress : warnings.simplefilter('ignore', category=IgnoredArgumentWarning)          │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [66]:
results[0]

,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast
2000-01-06,4.0,3.0,2.0,1.0,0.0,0
2000-01-07,5.0,4.0,3.0,2.0,1.0,0
2000-01-08,6.0,5.0,4.0,3.0,2.0,0
2000-01-09,7.0,6.0,5.0,4.0,3.0,0
2000-01-10,8.0,7.0,6.0,5.0,4.0,0
2000-01-06,4.0,3.0,2.0,1.0,0.0,1
2000-01-07,5.0,4.0,3.0,2.0,1.0,1
2000-01-08,6.0,5.0,4.0,3.0,2.0,1
2000-01-09,7.0,6.0,5.0,4.0,3.0,1
2000-01-10,8.0,7.0,6.0,5.0,4.0,1


In [77]:
series_dict = {
    'series_1': pd.Series(np.arange(50), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_2': pd.Series(np.arange(10, 40), index=pd.RangeIndex(start=20, stop=50, step=1)),
    'series_3': pd.Series(np.arange(20, 70), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_4': None
}

In [78]:
cv = TimeSeriesFold(
            initial_train_size = 30,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,window_size=forecaster.window_size, verbose=False
         )

In [79]:
cv._extract_index(X=series_dict)

AttributeError: 'NoneType' object has no attribute 'index'

In [17]:
series = pd.DataFrame({'1': pd.Series(np.arange(10, dtype=float)), 
                           '2': pd.Series(np.arange(10, dtype=float))})
series.index = pd.date_range(start='2000-01-01', periods=len(series), freq='D')
series = series_wide_to_long(series)
exog = pd.DataFrame({'exog_1': np.arange(100, 110, dtype=float),
                        'exog_2': np.arange(1000, 1010, dtype=float)})
exog.index = pd.date_range(start='2000-01-01', periods=len(exog), freq='D')
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"{i}") for i in range(1, 3)]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])

In [22]:
forecaster = ForecasterRecursiveMultiSeries(
    LinearRegression(), lags=5, encoding='ordinal_category', transformer_series=None
)
results = forecaster._create_train_X_y(
    series=series, exog=exog, store_last_window=False
)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:478                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [27]:
series = pd.DataFrame({'l1': pd.Series(np.arange(10, dtype=float)), 
                           'l2': pd.Series([np.nan, np.nan, 2., 3., 4., 5., 6., 7., 8., 9.])})
series.index = pd.date_range("1990-01-01", periods=10, freq='D')
series = series_wide_to_long(series)
series

value
series_id datetime         
l1        1990-01-01    0.0
          1990-01-02    1.0
          1990-01-03    2.0
          1990-01-04    3.0
          1990-01-05    4.0
          1990-01-06    5.0
          1990-01-07    6.0
          1990-01-08    7.0
          1990-01-09    8.0
          1990-01-10    9.0
l2        1990-01-01    NaN
          1990-01-02    NaN
          1990-01-03    2.0
          1990-01-04    3.0
          1990-01-05    4.0
          1990-01-06    5.0
          1990-01-07    6.0
          1990-01-08    7.0
          1990-01-09    8.0
          1990-01-10    9.0

In [33]:
series = {
    'l1': pd.Series(np.arange(10, dtype=float)), 
    'l2': pd.Series(np.arange(10, dtype=float))
}
series['l1'].loc[3] = np.nan
series['l1'].index = pd.date_range("1990-01-01", periods=10, freq='D')
series['l2'].index = pd.date_range("1990-01-01", periods=10, freq='D')

exog = pd.Series(np.arange(100, 110), name='exog', dtype=float)
l1_index = np.repeat(list(series.keys()), len(exog))
l2_index = np.tile(pd.date_range("1990-01-01", periods=10, freq='D'), len(series.keys()))
multi_index = pd.MultiIndex.from_arrays([l1_index, l2_index], names=["series_id", "datetime"])
exog = pd.Series(np.tile(exog.to_numpy(), len(series.keys())), index=multi_index, name="exog")
exog

series_id  datetime  
l1         1990-01-01    100.0
           1990-01-02    101.0
           1990-01-03    102.0
           1990-01-04    103.0
           1990-01-05    104.0
           1990-01-06    105.0
           1990-01-07    106.0
           1990-01-08    107.0
           1990-01-09    108.0
           1990-01-10    109.0
l2         1990-01-01    100.0
           1990-01-02    101.0
           1990-01-03    102.0
           1990-01-04    103.0
           1990-01-05    104.0
           1990-01-06    105.0
           1990-01-07    106.0
           1990-01-08    107.0
           1990-01-09    108.0
           1990-01-10    109.0
Name: exog, dtype: float64

In [32]:
l1_index

array([dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2']),
       dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2']),
       dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2']),
       dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2']),
       dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2'])], dtype=object)

In [8]:
series_2 = {
    "series_1": pd.Series(np.arange(15, dtype=float)),
    "series_2": pd.Series(np.arange(50, 65, dtype=float)),
}
exog_2 = pd.DataFrame(
    {
        "exog_1": np.arange(100, 115, dtype=float),
        "exog_2": np.arange(1000, 1015, dtype=int),
    },
)
exog_2 = {"series_1": exog_2, "series_2": exog_2}

forecaster = ForecasterRecursiveMultiSeries(LinearRegression(), lags=5)
X_train, y_train, X_test, y_test, X_train_encoding, X_test_encoding = (
    forecaster._train_test_split_one_step_ahead(
        series=series_2, exog=exog_2, initial_train_size=10
    )
)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:478                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

5
Index([5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')


TypeError: If `series` is a dictionary, all series must have a Pandas RangeIndex or DatetimeIndex with the same step/frequency. Review series: ['series_1', 'series_2']

In [9]:
series_2 = {
    "series_1": pd.Series(np.arange(15, dtype=float)),
    "series_2": pd.Series(np.arange(50, 65, dtype=float)),
}

In [4]:
series_dict

{'series_1': datetime
 2023-01-01    64
 2023-01-02    55
 2023-01-03    59
 2023-01-04     2
 2023-01-05    20
               ..
 2036-09-04    28
 2036-09-05    89
 2036-09-06    68
 2036-09-07    80
 2036-09-08    52
 Freq: D, Name: value, Length: 5000, dtype: int32,
 'series_2': datetime
 2023-01-01    34
 2023-01-02    69
 2023-01-03    85
 2023-01-04    17
 2023-01-05    74
               ..
 2036-09-04    19
 2036-09-05     8
 2036-09-06    16
 2036-09-07    41
 2036-09-08    77
 Freq: D, Name: value, Length: 5000, dtype: int32,
 'series_3': datetime
 2023-01-01    95
 2023-01-02    90
 2023-01-03    35
 2023-01-04    86
 2023-01-05    81
               ..
 2036-09-04    64
 2036-09-05    89
 2036-09-06    90
 2036-09-07    30
 2036-09-08    43
 Freq: D, Name: value, Length: 5000, dtype: int32,
 'series_4': datetime
 2023-01-01    56
 2023-01-02    90
 2023-01-03    70
 2023-01-04    63
 2023-01-05    57
               ..
 2036-09-04    84
 2036-09-05    21
 2036-09-06    76
 20

In [9]:
pd.date_range(start='2000-01-01', periods=50, freq='D')

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14', '2000-01-15', '2000-01-16',
               '2000-01-17', '2000-01-18', '2000-01-19', '2000-01-20',
               '2000-01-21', '2000-01-22', '2000-01-23', '2000-01-24',
               '2000-01-25', '2000-01-26', '2000-01-27', '2000-01-28',
               '2000-01-29', '2000-01-30', '2000-01-31', '2000-02-01',
               '2000-02-02', '2000-02-03', '2000-02-04', '2000-02-05',
               '2000-02-06', '2000-02-07', '2000-02-08', '2000-02-09',
               '2000-02-10', '2000-02-11', '2000-02-12', '2000-02-13',
               '2000-02-14', '2000-02-15', '2000-02-16', '2000-02-17',
               '2000-02-18', '2000-02-19'],
              dtype='datetime64[ns]', freq='D')

In [10]:
series = pd.DataFrame(
             {'1': pd.Series(np.array(
                       [0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
                        0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
                        0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
                        0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
                        0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
                        0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
                        0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
                        0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
                        0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
                        0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453])
                   ), 
              '2': pd.Series(np.array(
                       [0.12062867, 0.8263408 , 0.60306013, 0.54506801, 0.34276383,
                        0.30412079, 0.41702221, 0.68130077, 0.87545684, 0.51042234,
                        0.66931378, 0.58593655, 0.6249035 , 0.67468905, 0.84234244,
                        0.08319499, 0.76368284, 0.24366637, 0.19422296, 0.57245696,
                        0.09571252, 0.88532683, 0.62724897, 0.72341636, 0.01612921,
                        0.59443188, 0.55678519, 0.15895964, 0.15307052, 0.69552953,
                        0.31876643, 0.6919703 , 0.55438325, 0.38895057, 0.92513249,
                        0.84167   , 0.35739757, 0.04359146, 0.30476807, 0.39818568,
                        0.70495883, 0.99535848, 0.35591487, 0.76254781, 0.59317692,
                        0.6917018 , 0.15112745, 0.39887629, 0.2408559 , 0.34345601])
                   )
             }
         )
series.index = pd.date_range(start='2000-01-01', periods=len(series), freq='D')

series_2 = series.copy()
series_2.columns = ['l1', 'l2']

series_as_dict = series_2.copy().to_dict(orient='series')
series_long = series_wide_to_long(series)
series_long

value
series_id datetime            
1         2000-01-01  0.696469
          2000-01-02  0.286139
          2000-01-03  0.226851
          2000-01-04  0.551315
          2000-01-05  0.719469
...                        ...
2         2000-02-15  0.691702
          2000-02-16  0.151127
          2000-02-17  0.398876
          2000-02-18  0.240856
          2000-02-19  0.343456

[100 rows x 1 columns]

In [12]:
series = pd.DataFrame(
             {'1': pd.Series(np.array(
                       [0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
                        0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
                        0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
                        0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
                        0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
                        0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
                        0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
                        0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
                        0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
                        0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453])
                   ), 
              '2': pd.Series(np.array(
                       [0.12062867, 0.8263408 , 0.60306013, 0.54506801, 0.34276383,
                        0.30412079, 0.41702221, 0.68130077, 0.87545684, 0.51042234,
                        0.66931378, 0.58593655, 0.6249035 , 0.67468905, 0.84234244,
                        0.08319499, 0.76368284, 0.24366637, 0.19422296, 0.57245696,
                        0.09571252, 0.88532683, 0.62724897, 0.72341636, 0.01612921,
                        0.59443188, 0.55678519, 0.15895964, 0.15307052, 0.69552953,
                        0.31876643, 0.6919703 , 0.55438325, 0.38895057, 0.92513249,
                        0.84167   , 0.35739757, 0.04359146, 0.30476807, 0.39818568,
                        0.70495883, 0.99535848, 0.35591487, 0.76254781, 0.59317692,
                        0.6917018 , 0.15112745, 0.39887629, 0.2408559 , 0.34345601])
                   )
             }
         )

In [15]:
exog_wide_dt = pd.DataFrame(
           {'exog_1': pd.Series(np.array(
                          [0.51312815, 0.66662455, 0.10590849, 0.13089495, 0.32198061,
                           0.66156434, 0.84650623, 0.55325734, 0.85445249, 0.38483781,
                           0.3167879 , 0.35426468, 0.17108183, 0.82911263, 0.33867085,
                           0.55237008, 0.57855147, 0.52153306, 0.00268806, 0.98834542,
                           0.90534158, 0.20763586, 0.29248941, 0.52001015, 0.90191137,
                           0.98363088, 0.25754206, 0.56435904, 0.80696868, 0.39437005,
                           0.73107304, 0.16106901, 0.60069857, 0.86586446, 0.98352161,
                           0.07936579, 0.42834727, 0.20454286, 0.45063649, 0.54776357,
                           0.09332671, 0.29686078, 0.92758424, 0.56900373, 0.457412  ,
                           0.75352599, 0.74186215, 0.04857903, 0.7086974 , 0.83924335])
                      ),
            'exog_2': ['a'] * 25 + ['b'] * 25}
       )
exog_wide_dt.index = pd.date_range(start='2000-01-01', periods=len(exog_wide_dt), freq='D')

exog_long_dt = series_wide_to_long(exog_wide_dt)
exog_long_dt

value
series_id datetime            
exog_1    2000-01-01  0.513128
          2000-01-02  0.666625
          2000-01-03  0.105908
          2000-01-04  0.130895
          2000-01-05  0.321981
...                        ...
exog_2    2000-02-15         b
          2000-02-16         b
          2000-02-17         b
          2000-02-18         b
          2000-02-19         b

[100 rows x 1 columns]

In [42]:
transformer_exog = ColumnTransformer(
                           [('scale', StandardScaler(), ['exog_1']),
                            ('onehot', OneHotEncoder(), ['exog_2'])],
                           remainder = 'passthrough',
                           verbose_feature_names_out = False
                       )
forecaster = ForecasterRecursiveMultiSeries(
                    regressor          = LinearRegression(),
                    lags               = 5,
                    transformer_series = StandardScaler(),
                    transformer_exog   = transformer_exog,
                )
forecaster.fit(series=series_dict_dt, exog=exog_long_dt)

In [43]:
print(
    forecaster.exog_in_,
    forecaster.exog_names_in_,
    forecaster.exog_type_in_,
    forecaster.exog_dtypes_in_,
    forecaster.exog_dtypes_out_ ,
    forecaster.X_train_exog_names_out_ 
)

True ['exog_1', 'exog_2'] <class 'pandas.core.frame.DataFrame'> {'exog_1': dtype('float64'), 'exog_2': dtype('O')} {'exog_1': dtype('float64'), 'exog_2_a': dtype('float64'), 'exog_2_b': dtype('float64')} ['exog_1', 'exog_2_a', 'exog_2_b']


In [46]:

results = forecaster._create_predict_inputs(
    steps=5, levels='l1', exog=exog_pred_long_dt
)

In [47]:
results = forecaster.create_predict_X(steps=5, exog=exog_pred_dict_range)

TypeError: Expected index of type <class 'pandas.core.indexes.datetimes.DatetimeIndex'> for `exog` for series 'l1'. Got <class 'pandas.core.indexes.range.RangeIndex'>.

In [38]:
exog_wide_range = pd.DataFrame(
    {'exog_1': pd.Series(np.array(
                    [0.51312815, 0.66662455, 0.10590849, 0.13089495, 0.32198061,
                    0.66156434, 0.84650623, 0.55325734, 0.85445249, 0.38483781,
                    0.3167879 , 0.35426468, 0.17108183, 0.82911263, 0.33867085,
                    0.55237008, 0.57855147, 0.52153306, 0.00268806, 0.98834542,
                    0.90534158, 0.20763586, 0.29248941, 0.52001015, 0.90191137,
                    0.98363088, 0.25754206, 0.56435904, 0.80696868, 0.39437005,
                    0.73107304, 0.16106901, 0.60069857, 0.86586446, 0.98352161,
                    0.07936579, 0.42834727, 0.20454286, 0.45063649, 0.54776357,
                    0.09332671, 0.29686078, 0.92758424, 0.56900373, 0.457412  ,
                    0.75352599, 0.74186215, 0.04857903, 0.7086974 , 0.83924335])
                ),
    'exog_2': ['a'] * 25 + ['b'] * 25}
)
exog_wide_dt = exog_wide_range.copy()
exog_wide_dt.index = pd.date_range(start='2000-01-01', periods=len(exog_wide_dt), freq='D')

exog_long_dt = exog_wide_dt.copy()
exog_long_dt.index.name = "datetime"
exog_long_dt = [exog_long_dt.assign(series_id=f"l{i}") for i in range(1, 3)]
exog_long_dt = pd.concat(exog_long_dt)
exog_long_dt = exog_long_dt.set_index(["series_id", exog_long_dt.index])
exog_long_dt

exog_1 exog_2
series_id datetime                   
l1        2000-01-01  0.513128      a
          2000-01-02  0.666625      a
          2000-01-03  0.105908      a
          2000-01-04  0.130895      a
          2000-01-05  0.321981      a
...                        ...    ...
l2        2000-02-15  0.753526      b
          2000-02-16  0.741862      b
          2000-02-17  0.048579      b
          2000-02-18  0.708697      b
          2000-02-19  0.839243      b

[100 rows x 2 columns]

In [48]:
{
        'exog_1': exog_wide_dt['exog_1'].dtype, 
        'exog_2': exog_wide_dt['exog_2'].dtype
    }

{'exog_1': dtype('float64'), 'exog_2': dtype('O')}

In [59]:
series_dict_range_nan = {
        '1': series_dict_range['l1'].copy(),
        '2': series_dict_range['l2'].copy()
    }
series_dict_range_nan['1'].iloc[:10] = np.nan

transformer_exog = ColumnTransformer(
                        [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                        remainder = 'passthrough',
                        verbose_feature_names_out = False
                    )
forecaster = ForecasterRecursiveMultiSeries(
                    regressor          = LinearRegression(),
                    lags               = 5,
                    transformer_series = StandardScaler(),
                    transformer_exog   = transformer_exog,
                )
forecaster.fit(series=series_dict_range_nan, exog=exog_wide_range)
results = forecaster.create_predict_X(steps=5, exog=exog_pred_wide_range)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ The output matrix is in the transformed scale due to the inclusion of                │
│ transformations or differentiation in the Forecaster. As a result, any predictions   │
│ generated using this matrix will also be in the transformed scale. Please refer to   │
│ the documentation for more details:                                                  │
│ https://skforecast.org/latest/user_guides/training-and-prediction-matrices.html      │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:2460                                                                         │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [60]:
from skforecast.preprocessing import TimeSeriesDifferentiator

In [61]:
end_train = 30
series_dict_range_train = {
    'l1': series_dict_range['l1'].loc[:end_train],
    'l2': series_dict_range['l2'].loc[:end_train]
}
# Data differentiated
differentiator = TimeSeriesDifferentiator(order=1)
series_1_diff = differentiator.fit_transform(series_dict_range['l1'].to_numpy())
series_2_diff = differentiator.fit_transform(series_dict_range['l2'].to_numpy())
data_diff = {
    'l1': pd.Series(series_1_diff[1:]), 
    'l2': pd.Series(series_2_diff[1:])
}
data_diff['l1'].index = series_dict_range['l1'].index[1:]
data_diff['l2'].index = series_dict_range['l2'].index[1:]
data_diff_train = {
    'l1': data_diff['l1'].loc[:end_train],
    'l2': data_diff['l2'].loc[:end_train]
}

# Simulated exogenous variable
rng = np.random.default_rng(9876)
exog = pd.Series(
    rng.normal(loc=0, scale=1, size=len(series_dict_range['l1'])), 
    index=series_dict_range['l1'].index, name='exog'
)
exog_diff = exog.iloc[1:]

steps = len(data_diff['l1'].loc[end_train + 1:])

forecaster_1 = ForecasterRecursiveMultiSeries(
    regressor=LinearRegression(), lags=15, transformer_series=None
)
forecaster_1.fit(series=data_diff_train, exog=exog_diff.loc[:end_train])
predictions_diff = forecaster_1.predict(steps=steps, exog=exog_diff.loc[end_train + 1:])
# Revert the differentiation
predictions_diff = pd.pivot_table(
    predictions_diff, index=predictions_diff.index, columns="level", values="pred"
)
last_value_train = pd.DataFrame(data_diff_train).iloc[[-1]]

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:478                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [62]:
last_value_train

,l1,l2
30,-0.538871,-0.376763


In [39]:
exog_pred_long_dt

exog_1 exog_2
series_id datetime                   
l1        2000-02-20  0.513128      a
          2000-02-21  0.666625      a
          2000-02-22  0.105908      a
          2000-02-23  0.130895      a
          2000-02-24  0.321981      a
...                        ...    ...
l2        2000-04-05  0.753526      b
          2000-04-06  0.741862      b
          2000-04-07  0.048579      b
          2000-04-08  0.708697      b
          2000-04-09  0.839243      b

[100 rows x 2 columns]

In [63]:
end_train = 30
series_dict_range_train = {
    'l1': series_dict_range['l1'].loc[:end_train],
    'l2': series_dict_range['l2'].loc[:end_train]
}
# Data differentiated
differentiator = TimeSeriesDifferentiator(order=1)
series_1_diff = differentiator.fit_transform(series_dict_range['l1'].to_numpy())
series_2_diff = differentiator.fit_transform(series_dict_range['l2'].to_numpy())
data_diff = {
    'l1': pd.Series(series_1_diff[1:]), 
    'l2': pd.Series(series_2_diff[1:])
}
data_diff['l1'].index = series_dict_range['l1'].index[1:]
data_diff['l2'].index = series_dict_range['l2'].index[1:]
data_diff['l1'].index

RangeIndex(start=1, stop=50, step=1)